In [1]:
import numpy as np
# import cupy as cp
import matplotlib.pyplot as plt
import matplotlib
from scipy.ndimage import convolve
matplotlib.use('TkAgg')  # Usar TkAgg backend para ventanas externas

In [ ]:
def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
   
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', 'checkerboard', 'cruz', 'linea')
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
   
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return np.random.randint(0, m+1, grid_size)
   
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
       
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
   
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = np.indices(grid_size)
        sum_indices = np.sum(indices, axis=0)
        return (sum_indices % 2) * m
        
    elif pattern == 'cross':
        # Patrón de cruz - funciona mejor en 2D o más dimensiones
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Para cada dimensión, crear una línea a través del centro
        for dim in range(d):
            # Crear índices para la línea en esta dimensión
            indices = [slice(0, s) if i != dim else center[i] for i, s in enumerate(grid_size)]
            grid[tuple(indices)] = m
        
        return grid
        
    elif pattern == 'line':
        # Patrón de línea horizontal (a lo largo de la primera dimensión)
        grid = np.zeros(grid_size, dtype=int)
        
        # En numpy, la primera dimensión normalmente se considera como filas (eje y)
        # Para una línea horizontal, necesitamos fijar la coordenada y y variar x
        if d >= 2:  # Asegurar que tenemos al menos 2 dimensiones para una línea horizontal
            # Crear índices para una línea horizontal
            # Fijamos la segunda dimensión (normalmente el eje y) en el centro
            indices = [grid_size[1] // 2, slice(0, grid_size[0])]
            
            # Para dimensiones adicionales, elegimos el punto central
            for dim in range(2, d):
                indices.append(grid_size[dim] // 2)
            
            # En numpy, los índices se dan en orden [y, x, z, ...]
            grid[tuple(indices)] = m
        else:  # Para el caso unidimensional
            grid[:] = m
        
        return grid
   
    else:
        # Patrón predeterminado: valores 0
        return np.zeros(grid_size, dtype=int)

In [ ]:
def crear_kernel_euclidiano(dimensiones, radio):
    """
    Crea un kernel euclidiano (circular en 2D, esférico en 3D, etc.) para la vecindad.
    
    Parámetros:
    - dimensiones: Número de dimensiones del espacio.
    - radio: Radio de la vecindad.
    
    Retorna:
    - kernel: Array de numpy con 1s donde hay vecindad y 0s donde no.
    """
    # Construir el kernel para una vecindad euclidiana
    kernel_shape = (2 * radio + 1,) * dimensiones
    kernel_size = np.prod(kernel_shape)
    
    # Usar reshape para generar las coordenadas de manera eficiente
    linear_indices = np.arange(kernel_size).reshape(kernel_shape)
    
    # Calcular coordenadas centrales
    center_coords = np.array([radio] * dimensiones)
    
    # Convertir índices lineales a coordenadas
    coords = np.unravel_index(linear_indices.flatten(), kernel_shape)
    coords = np.array(coords).reshape(dimensiones, -1).T
    
    # Calcular distancias al cuadrado al centro
    squared_distances = np.sum((coords - center_coords)**2, axis=1)
    
    # Crear kernel con forma euclidiana usando reshape
    kernel = (squared_distances <= radio**2).astype(np.int32)
    kernel = kernel.reshape(kernel_shape)
    
    # Localizar y excluir la celda central
    center_index = tuple(radio for _ in range(dimensiones))
    kernel[center_index] = 0
    
    return kernel

In [ ]:
def expandir_grilla(grilla, radio):
    """
    Expande la grilla con bordes periódicos (toroide).
    
    Parámetros:
    - grilla: ndarray de numpy con la configuración actual.
    - radio: Radio de vecindad que determina cuánto expandir.
    
    Retorna:
    - grilla_expandida: Grilla con bordes periódicos.
    """
    return np.pad(grilla, radio, mode='wrap')

In [ ]:
def calcular_suma_vecinos(grilla_expandida, kernel):
    """
    Calcula la suma de valores en la vecindad para cada celda.
    
    Parámetros:
    - grilla_expandida: Grilla con bordes expandidos.
    - kernel: Kernel que define la vecindad.
    
    Retorna:
    - suma_vecinos: Array con la suma de valores vecinos para cada celda.
    """
    # Convolución para obtener la suma de vecinos
    suma_vecinos = convolve(grilla_expandida, kernel, mode='wrap')
    # Extraer la parte válida de la convolución
    radio = kernel.shape[0] // 2
    slices = tuple(slice(radio, -radio) for _ in range(grilla_expandida.ndim))
    suma_vecinos = suma_vecinos[slices]
    
    return suma_vecinos

In [ ]:
def calcular_intervalos(kernel, m):
    """
    Calcula los intervalos para aplicar las reglas de evolución.
    
    Parámetros:
    - kernel: Kernel que define la vecindad.
    - m: Número máximo de estados por celda.
    
    Retorna:
    - intervalos: Array con los límites de los 3 intervalos.
    """
    num_vecinos = np.sum(kernel)
    SM = m * num_vecinos
    intervalos = np.linspace(0, SM, 4)  # Dividimos en 3 intervalos
    return intervalos

In [ ]:

def aplicar_reglas_evolucion(grilla, suma_vecinos, intervalos, m):
    """
    Aplica las reglas de evolución al autómata celular.
    
    Parámetros:
    - grilla: Configuración actual del autómata.
    - suma_vecinos: Suma de valores en la vecindad para cada celda.
    - intervalos: Límites de los intervalos para aplicar reglas.
    - m: Número máximo de estados por celda.
    
    Retorna:
    - nueva_grilla: Grilla evolucionada según las reglas.
    """
    # Copia de la grilla para evitar modificar la original
    nueva_grilla = grilla.copy()
    
    # Aplicar reglas con máscaras
    mask1 = (suma_vecinos >= intervalos[0]) & (suma_vecinos < intervalos[1])
    mask2 = (suma_vecinos >= intervalos[1]) & (suma_vecinos < intervalos[2])
    mask3 = (suma_vecinos >= intervalos[2])

    # Modificaciones en orden para evitar conflictos
    nueva_grilla[mask1] = np.maximum(nueva_grilla[mask1] - 1, 0)  # Evita negativos
    nueva_grilla[mask2] = np.minimum(nueva_grilla[mask2] + 1, m)  # No sobrepasa m
    nueva_grilla[mask3] = np.maximum(nueva_grilla[mask3] - 1, 0)  # Evita negativos

    return nueva_grilla